In [4]:
from sklearn.metrics.pairwise import polynomial_kernel

In [1]:
from feature_utils.parallel import BufferedSOAPFeatures, get_features_in_parallel, get_optimal_radial_basis_hypers_parallel
from loader.loader import load_data
from copy import deepcopy
import numpy as np
from sklearn.linear_model import RidgeCV
from rascal.representations import SphericalInvariants as SOAP
from rascal.utils import get_optimal_radial_basis_hypers
from rascal.neighbourlist.structure_manager import mask_center_atoms_by_id
from skcosmo.model_selection import atom_groups_by_frame
from sklearn.linear_model import LinearRegression, Ridge
from copy import deepcopy
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import time
from sklearn.compose import TransformedTargetRegressor
from skcosmo.preprocessing import StandardFlexibleScaler
#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn.svm import LinearSVR
from sklearn.base import clone
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process.kernels import PairwiseKernel

In [2]:
from skcosmo.preprocessing import StandardFlexibleScaler

In [37]:
train_structures, test_structures, train_properties, test_properties = load_data("./make_tensor_data/train_tensor/CSD-3k+S546_shift_tensors.xyz",\
                                                                                    "./make_tensor_data/test_tensor/CSD-500+104-7_shift_tensors.xyz",selected_species=7, random_subsample_train=10)

In [35]:
from rascal.neighbourlist.structure_manager import AtomsList, get_neighbourlist

In [38]:
len(test_properties)

3367

In [25]:
get_neighbourlist(train_structures,)

TypeError: get_neighbourlist() missing 1 required positional argument: 'options'

In [11]:
this = AtomsList(train_structures,dict(name="centers", args=dict()))

TypeError: string indices must be integers

In [12]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=4.5,
              max_radial=8,
              max_angular=8,
              gaussian_sigma_constant=0.3,
              gaussian_sigma_type="Constant",
              radial_basis="GTO",
              normalize=True,
              cutoff_smooth_width=0.3,
              cutoff_function_type="RadialScaling",
              optimization=
                    dict(
                            Spline=dict(
                               accuracy=1.0e-05
                            )
                        ),
              cutoff_function_parameters= dict(rate=1,
                    scale=2.0,
                    exponent=3
                        ),
              compute_gradients=False,
              expansion_by_species_method="user defined",
              global_species=[1, 6, 7, 8, 16]
              )

In [20]:
feat = SOAP(**hypers).transform(train_structures)


In [33]:
feat.get_distances().shape

(96462,)

In [17]:
BufferedSOAPFeatures.get_features(hypers)

TypeError: get_features() missing 1 required positional argument: 'update_params'

In [12]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=4.5,
              max_radial=8,
              max_angular=8,
              gaussian_sigma_constant=0.3,
              gaussian_sigma_type="Constant",
              radial_basis="GTO",
              normalize=True,
              cutoff_smooth_width=0.3,
              cutoff_function_type="RadialScaling",
              optimization=
                    dict(
                            Spline=dict(
                               accuracy=1.0e-05
                            )
                        ),
              cutoff_function_parameters= dict(rate=1,
                    scale=2.0,
                    exponent=3
                        ),
              compute_gradients=False,
              expansion_by_species_method="user defined",
              global_species=[1, 6, 7, 8, 16]
              )

hypers = get_optimal_radial_basis_hypers_parallel(hypers,train_structures)

nested_param = dict(rate=1,
                    scale=3.0,
                    exponent=6
                        )

kernel_params = dict()

"""space = [Real(10**-5, 10**2, "log-uniform", name='alpha'),
        Real(0.05,1.5, "uniform", name="gaussian_sigma_constant"),
        Real(2.,4.5, "uniform", name="interaction_cutoff")]"""

space = [Real(10**-6, 10**1, "log-uniform", name='alpha'),
        #Real(1.,3.5, "uniform", name="scale"),
        Real(3.5,8.0, "uniform", name="interaction_cutoff"),
        Real(0.01,0.5, "uniform", name="gaussian_sigma_constant"),
        #Real(1.,5., "uniform", name="exponent"),
        #Real(0.5,3., "uniform", name="rate")
        ]

reg = Ridge()
#TransformedTargetRegressor(regressor=LinearSVR(loss="squared_epsilon_insensitive"), transformer=StandardFlexibleScaler(),check_inverse=False)
y = train_properties
atom_groups = atom_groups_by_frame(train_structures)
Feature_gen = BufferedSOAPFeatures(train_structures, hypers,optimize=False,n_cores=-1)




@use_named_args(space)
def soap_objective(**params):
    update_dict = {}
    
    new_params = params.copy()
    #nested_dict_param = deepcopy(Feature_gen.calculator_params)
    #nested_dict_param = nested_dict_param["cutoff_function_parameters"]
    
    for key, value in new_params.items():
        if key in Feature_gen.calculator_params:
            #hypers[key] = value
            update_dict[key] = params.pop(key, None)
        """if key in nested_param:
            nested_dict_param[key] = params.pop(key, None)
            #print("I update my {} to {}".format(key,value))
        
        update_dict["cutoff_function_parameters"] = nested_dict_param"""
            
            
    #print(update_dict)
    start_time = time.time()
    
    
    
    X = Feature_gen.get_features(update_dict)
    print("--- feature_gen time %s seconds ---" % (time.time() - start_time))
    #print(X.shape)
    
    start_time = time.time()
    Kernel = polynomial_kernel(X)
    #print(Feature_gen.hypers["max_angular"])
    start_time = time.time()
    splits = list(GroupKFold(n_splits=3).split(Kernel,y,groups=atom_groups))
    
    score = [] 
    
    reg.set_params({"alpha":params.pop("alpha",None)})
    
    
    
    Kernel = 
    
    
    for trainsplit, testsplit in GroupKFold(n_splits=5).split(X,y,groups=atom_groups):
        X_train = X[trainsplit]
        X_test = X[testsplit]

        
        y_train = y[trainsplit].reshape(-1, 1)
        #print(y_train)
        y_test = y[testsplit].reshape(-1, 1)

        scaler_X = StandardFlexibleScaler().fit(X_train)
        X_train = scaler_X.transform(X_train,copy=True)
        X_test = scaler_X.transform(X_test,copy=True)

        scaler_y = StandardFlexibleScaler().fit(y_train)
        y_train = scaler_y.transform(y_train,copy=True)
        y_test = scaler_y.transform(y_test,copy=True)

        reg.fit(X_train,y_train)

        y_test_pred = reg.predict(X_test)

        score.append(mean_squared_error(scaler_y.inverse_transform(y_test), scaler_y.inverse_transform(y_test_pred)))
        
    #print(reg.alpha)
    
    #score = -np.mean(cross_val_score(reg, X, y, cv=splits, n_jobs=1,
    #                                scoring="neg_mean_squared_error"))
    
    print("--- cross_val time %s seconds ---" % (time.time() - start_time))
    #print(score)    
    return -np.mean(score)